In [66]:
# Import necessary libraries and modules
%matplotlib widget
import numpy as np
import scipy.constants as scp
import math
import matplotlib.pyplot as plt
import Photon_Class as P  # Photon_Class contains the definition of the Photon class
import Sim_data as data # Sim_data contains definiations for QE and abs data
import random
import csv
import MonteCarlo_V2 as MC  # MonteCarlo_V2 contains the definition of the start_sim function
from random import seed


# Tank Constants (cube)
width, height = 1, 3 #m
n_water = 1.33
tank_boundaries = np.array([[0, width], [0, height], [0, width]])    
atl_water = 0.0002 #attenuation length cm
thicknesses = np.arange(1,26,1) #mm
sizes = np.arange(10,61,5) #cm
water = False


# Photon Constants
wavelength = 380 #nm
v_mag = scp.c * n_water  # Velocity magnitude in water


# PMT Constants
rx, ry, rz = 0.0762/2, 40e-3, 0.0762/2  # PMT radii
pmt_radii = np.array([rx, ry, rz])
cx, cy, cz = width/2, 0, width/2  # PMT center positions
pmt_center = np.array([cx, cy, cz]) #center of tank


# WLS Constants
n_wls = 1.58  # Refractive index of WLS plate
ca = math.asin(n_water/n_wls)  # Critical angle between WLS and water
atl_wls = 0.006  # Attenuation length
thicknesses = np.arange(1e-3,26e-3,1e-3) #mm
sizes = np.arange(10e-2,61e-2,5e-2) #cm
wls_center = np.array([width/2, 0, width/2]) #center of tank


# Simulation constants
nruns = 1000 #num of photons

# Create a header list for the output CSV file
header_list = ['Run', 'Initial_wl_nm', 'Absorption_length_m', 'Absorption_coordinate_m', 
               'Isotropic_emmission_angle_theta_deg', 'Isotropic_emmission_angle_phi_deg', 'Shifted_wl_nm',
               'Refraction_coordinate_m', 'Hit_coordinate_m', 'Detected',
               'Num_reflections', 'Quantumn_efficiency', 'X_positions_m', 
               'Y_positions_m', 'Z_positions_m', 'Time_s', 'Timed_out']


for s in sizes:

    for t in thicknesses:

        #seed(0) # same random seed for each run

        w, h, l = s*1e-2, t*1e-3, s*1e-2  # Dimensions for WLS plate
        wls_boundaries = np.array([[0, w], [0, h], [0, l]])  # Boundaries

            
        path = 'outputs/'
        out_file = path + 'sim_output_' + str(t) + 'mm_' + str(s) + 'cm' + '.txt'

        # ------------------------------------------- Begin Simulation: -----------------------------------------------


        # Open the output file and write the header
        with open(out_file, 'w', newline='') as file:
            writer = csv.writer(file,  delimiter=';')
            writer.writerow(header_list)

            # Run the simulation for each run
            for n in range(nruns):
                # print("- LOG -")
                # print("Run:", n)

                # Refresh these variables every run
                orig_pos = np.array([random.uniform(0,width),height,random.uniform(0,width)]) #x,y,z randomly across top of tank
                theta, phi = random.uniform(math.pi, 2*math.pi), random.uniform(0,math.pi/2) #random direction towards lower hemisphere
                velocity = np.array([v_mag * math.sin(theta) * math.cos(phi), v_mag * math.sin(theta) * math.sin(phi), v_mag * math.cos(theta)])  # Velocity origin directions 

                # Run the simulation using the start_sim function from MonteCarlo_V2
                # Start with the photons in the water
                (absl, abs_coords, theta_iso, phi_iso, shwl, refract_coords, hit_coords, detected, 
                reflects, pmtqe, xpos, ypos, zpos, t, timeout) = MC.start_sim(orig_pos, velocity, wavelength, atl_water, 
                                                                            tank_boundaries, pmt_center, pmt_radii, ca, water, wls_boundaries)

                # Write the results of the simulation to the CSV file
                writer.writerow([n, wavelength , absl , abs_coords, theta_iso, phi_iso, shwl, refract_coords, 
                                hit_coords, detected, reflects, pmtqe, xpos, ypos, zpos, t, timeout])



Total Cherenkov photons: 6.76e+04
